In [68]:
# libraries
import random
import numpy as np
import pickle
import json
from flask import Flask, render_template, request
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


# chat initialization
model = load_model("model_output/chatbot_model.h5")
intents = json.loads(open("data/intents.json").read())
words = pickle.load(open("model_output/words.pkl", "rb"))
classes = pickle.load(open("model_output/classes.pkl", "rb"))

app = Flask(__name__)
# run_with_ngrok(app) 

@app.route("/")
def home():
    return render_template("index.html")


@app.route("/get", methods=["POST"])
def chatbot_response():
    msg = request.form["msg"].lower()
    print(msg)
    # if msg.startswith('hi'):
    #     name = msg[2:]
    #     ints = predict_class(msg, model)
    #     res1 = getResponse(ints, intents)
    #     res =res1.replace("{n}",name)
    # elif msg.startswith('hi my name is'):
    #     name = msg[14:]
    #     ints = predict_class(msg, model)
    #     res1 = getResponse(ints, intents)
    #     res =res1.replace("{n}",name)
    # else:
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


# chat functionalities
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    print(res)

    ERROR_THRESHOLD = 0.7

    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    #sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)

    confidence_probability=0.95
    return_list = []

    if len(results) == 0:
        return_list.append({"intent": 'failed', "probability": '1'})
    elif results[0][1] >=  confidence_probability:
        return_list.append({"intent": classes[results[0][0]], "probability": str(results[0][1])})
    else: 
        return_list.append({"intent": 'failed', "probability": '1'})
   
    print(return_list)
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    print(tag)
    list_of_intents = intents_json["intents"]

    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
        else:
            result= "Sorry, I don't understand that. Please rephrase/ask another question or send an email to gradanalytics@georgetown.edu"
    return result


if __name__ == "__main__":
    app.run(port='0000')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:53715
Press CTRL+C to quit
127.0.0.1 - - [03/Dec/2022 16:21:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 16:21:44] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2022 16:21:44] "GET /favicon.ico HTTP/1.1" 404 -


hi
1/1 [==============================] - 0s 60ms/step


2022-12-03 16:21:46.734438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
127.0.0.1 - - [03/Dec/2022 16:21:46] "POST /get HTTP/1.1" 200 -


[2.1651407e-05 1.3213247e-06 4.8463778e-07 1.1893407e-07 8.1770249e-06
 1.0409408e-07 7.6661047e-07 1.2377746e-04 7.9036397e-07 1.4257852e-05
 3.9663655e-06 7.8536635e-07 7.1298501e-07 1.0633505e-06 4.6595167e-05
 5.1433723e-05 9.9875641e-01 1.4912644e-06 1.9698833e-05 3.5121229e-05
 1.2936509e-07 4.8689642e-05 9.9173294e-06 5.0743543e-05 1.0032707e-05
 8.4062558e-05 8.7608396e-07 3.1078656e-05 9.7479119e-07 1.8432862e-05
 9.7988077e-06 2.4050460e-05 3.9294900e-05 8.6834643e-06 5.0162105e-04
 5.3833555e-05 1.9057195e-05]
[{'intent': 'greeting', 'probability': '0.9987564'}]
greeting
can i submit wes
1/1 [==============================] - 0s 16ms/step


...

[6.46908768e-04 6.09509480e-05 1.15004121e-04 6.67159256e-05
 2.58596090e-04 2.97391744e-05 6.36760305e-05 2.66899145e-03
 6.75965784e-05 1.52199878e-04 3.70801630e-04 8.72710661e-05
 1.18899465e-04 1.30275075e-04 3.17623769e-03 8.71454924e-03
 9.16591048e-01 1.04819417e-04 7.50531617e-04 9.25463508e-04
 1.50212854e-05 2.35312013e-03 2.47786054e-04 2.90682958e-03
 4.41492797e-04 2.81742611e-03 7.19098898e-05 2.45106057e-03
 3.19007930e-04 5.89159143e-04 5.78315696e-04 1.86685345e-03
 9.40358732e-04 3.02573346e-04 4.70437370e-02 1.14295969e-03
 8.12069047e-04]
[{'intent': 'failed', 'probability': '1'}]
failed
hellouuuuu
1/1 [==============================] - 0s 16ms/step


127.0.0.1 - - [03/Dec/2022 16:22:06] "POST /get HTTP/1.1" 200 -


[6.46908768e-04 6.09509480e-05 1.15004121e-04 6.67159256e-05
 2.58596090e-04 2.97391744e-05 6.36760305e-05 2.66899145e-03
 6.75965784e-05 1.52199878e-04 3.70801630e-04 8.72710661e-05
 1.18899465e-04 1.30275075e-04 3.17623769e-03 8.71454924e-03
 9.16591048e-01 1.04819417e-04 7.50531617e-04 9.25463508e-04
 1.50212854e-05 2.35312013e-03 2.47786054e-04 2.90682958e-03
 4.41492797e-04 2.81742611e-03 7.19098898e-05 2.45106057e-03
 3.19007930e-04 5.89159143e-04 5.78315696e-04 1.86685345e-03
 9.40358732e-04 3.02573346e-04 4.70437370e-02 1.14295969e-03
 8.12069047e-04]
[{'intent': 'failed', 'probability': '1'}]
failed
my name is nikita
1/1 [==============================] - 0s 16ms/step


127.0.0.1 - - [03/Dec/2022 16:22:17] "POST /get HTTP/1.1" 200 -


[9.8737970e-04 1.6448919e-04 3.9577702e-04 2.8201472e-04 1.0243540e-01
 8.2982320e-04 3.3515103e-03 7.0024985e-03 1.3348885e-03 4.6245050e-01
 8.0082379e-04 2.3643796e-03 2.3793388e-04 1.0596115e-02 4.9832324e-03
 8.0312032e-04 1.0427099e-01 1.1231845e-03 8.2463082e-03 7.0450653e-04
 1.8721379e-02 6.8905936e-03 1.7567861e-03 1.1045182e-03 4.0251637e-04
 4.3668557e-02 1.3099926e-03 4.5065422e-04 2.1984329e-04 4.7893482e-03
 8.1538968e-03 4.6355939e-03 3.5645671e-02 3.6012180e-04 6.3834077e-04
 1.4846669e-01 9.4206147e-03]
[{'intent': 'failed', 'probability': '1'}]
failed


In [64]:
results=[[16, 0.9987564]]
results[0][1]

0.9987564